##### RandomForestClassifier 分类
##### RandomForestRegressor 回归

需要调参的参数包括两部分：<br>第一部分是Bagging框架的参数，<br>第二部分是CART决策树的参数。

### 参数说明：
//classsklearn.ensemble.RandomForestClassifier(<br>
                            n_estimators=10, #弱学习器的最大迭代次数，或者说最大的弱学习器的个数，默认是10<br>
                            criterion='gini', 
                            max_depth=None,
                            min_samples_split=2, 
                            min_samples_leaf=1, 
                            min_weight_fraction_leaf=0.0,
                            max_features='auto', 
                            max_leaf_nodes=None, 
                            min_impurity_split=1e-07,
                            bootstrap=True, # 默认True，是否有放回的采样。<br>
                            oob_score=False, #默认false,是否采用袋外样本来评估模型的好坏
                            n_jobs=1, 
                            random_state=None, 
                            verbose=0,
                            warm_start=False, 
                            class_weight=None
                            )//
                            
### RF调参比较简单，因为弱分类器之间没有依赖关系

## Bagging框架的参数(RF重要的框架参数比较少，主要关注n_estimators，即RF最大的决策树个数。)

### n_estimators(子模型数): 弱学习器的最大迭代次数，或者说最大的弱学习器的个数，默认是10.<br>
1.n_estimators太小，容易欠拟合，n_estimators太大，又容易过拟合，<br>
2.增加“子模型数”（n_estimators）可以明显降低整体模型的方差，且不会对子模型的偏差和方差有任何影响<br>
3.模型的准确度会随着“子模型数”的增加而提高，由于减少的是整体模型方差公式的第二项，故准确度的提高有一个上限。在实际应用中，可以以10为单位，考察取值范围在1至201的调参情况。<br>

#### 与提升树对比：<br>
1.Random Forest:子模型都拥有*较低的偏差*，整体模型的训练过程旨在降低方差，故其需要较少的子模型（n_estimators默认值为10）且子模型不为弱模型（max_depth的默认值为None）<br>
2.Gradient Tree Boosting(提升树):子模型都拥有较低的方差，整体模型的训练过程旨在降低偏差，故其需要较多的子模型（n_estimators默认值为100）且子模型为弱模型（max_depth的默认值为3）

### bootstrap: # 默认True，是否有放回的采样。<br>
### oob_score: #默认false,是否采用袋外样本来评估模型的好坏<br>
有放回采样中大约36.8%的没有被采样到的数据，我们常常称之为袋外数据(Out Of Bag, 简称OOB)，这些数据没有参与训练集模型的拟合，因此可以用来检测模型的泛化能力。**个人推荐设置为True**，因为袋外分数反应了一个模型拟合后的泛化能力。对单个模型的参数训练，我们知道可以用cross validation（cv）来进行，但是特别消耗时间，而且对于随机森林这种情况也没有大的必要，所以就用这个数据对决策树模型进行验证，算是一个简单的交叉验证，性能消耗小，但是效果不错。

### criterion：
即CART树做划分时对特征的评价标准，<br>
#### 分类模型和回归模型的损失函数是**不一样**的:
分类RF对应的CART分类树默认是基尼系数gini,另一个可选择的标准是信息增益entropy，是用来选择节点的最优特征和切分点的两个准则。<br>
回归RF对应的CART回归树默认是均方差mse，另一个可以选择的标准是绝对值差mae。<br>
*一般来说选择默认的标准就已经很好的。*








## RF决策树的参数(要调参的参数基本和GBDT相同)
#### max_features: RF划分时考虑的最大特征数。可以使用很多种类型的值:
1.**默认是"None"**,意味着划分时考虑所有的特征数；<br>
2."log2":划分时最多考虑log2N个特征<br>
3."sqrt" or "auto":划分时最多考虑N−−√N个特征<br>
4.整数:考虑的特征绝对数。<br>
5.浮点数:考虑特征百分比，即考虑（百分比xN）取整后的特征数，其中N为样本总特征数。<br>
##### 一般来说，如果样本特征数不多，比如小于50，我们用默认的"None"就可以了，如果特征数非常多，我们可以灵活使用刚才描述的其他取值来控制划分时考虑的最大特征数，以控制决策树的生成时间。

#### max_depth: 决策树最大深度。默认为"None"，
决策树在建立子树的时候不会限制子树的深度, 这样建树时，会使每一个叶节点只有一个类别，或是达到min_samples_split。<br>
一般来说，数据少或者特征少的时候可以不管这个值。<br>
如果模型样本量多，特征也多的情况下，推荐限制这个最大深度，具体的取值取决于数据的分布。常用的可以取值10-100之间。

#### min_samples_split: 内部节点再划分所需最小样本数，默认2。这个值限制了子树继续划分的条件:
1.如果某节点的样本数少于min_samples_split，则不会继续再尝试选择最优特征来进行划分。<br> 
2.默认是2.如果样本量不大，不需要管这个值。如果样本量数量级非常大，则推荐增大这个值。

#### min_samples_leaf:叶子节点最少样本数。 这个值限制了叶子节点最少的样本数：
1.如果某叶子节点数目小于样本数，则会和兄弟节点一起被剪枝。 默认是1,可以输入最少的样本数的整数，或者最少样本数占样本总数的百分比。<br>
2.如果样本量不大，不需要管这个值。如果样本量数量级非常大，则推荐增大这个值。

#### min_weight_fraction_leaf：叶子节点最小的样本权重和。这个值限制了叶子节点所有样本权重和的最小值:
1.如果小于这个值，则会和兄弟节点一起被剪枝。 默认是0，就是不考虑权重问题。<br>
2.一般来说，如果我们有较多样本有缺失值，或者分类树样本的分布类别偏差很大，就会引入样本权重，这时我们就要注意这个值了。

#### max_leaf_nodes: 最大叶子节点数。通过限制最大叶子节点数，可以防止过拟合，默认是"None”，即不限制最大的叶子节点数。
1.如果加了限制，算法会建立在最大叶子节点数内最优的决策树。<br>
2.如果特征不多，可以不考虑这个值，但是如果特征分成多的话，可以加以限制，具体的值可以通过交叉验证得到。

#### min_impurity_split: 节点划分最小不纯度。这个值限制了决策树的增长:
1.如果某节点的不纯度(基于基尼系数，均方差)小于这个阈值，则该节点不再生成子节点，即为叶子节点 。<br>
2.一般不推荐改动默认值1e-7。


### 上面决策树参数中最重要的包括最大特征数max_features， 最大深度max_depth， 内部节点再划分所需最小样本数min_samples_split和叶子节点最少样本数min_samples_leaf。

#### splitter: 随机选择属性"random"还是选择不纯度最大"best"的属性，建议用默认 best。
#### presort:是否对数据进行预分类，以加快拟合中最佳分裂点的发现。默认False，适用于大数据集。
1.小数据集使用True,可以加快训练。<br>
2.是否预排序,预排序可以加速查找最佳分裂点，对于稀疏数据不管用，Bool，auto：非稀疏数据则预排序，若稀疏数据则不预排序


### 进行预测的几种常用方法
predict_proba(x)：给出带有概率值的结果。每个点在所有label（类别）的概率和为1. <br>
predict(x)：直接给出预测结果。内部还是调用的predict_proba()，根据概率的结果看哪个类型的预测值最高就是哪个类型。<br> 
predict_log_proba(x)：和predict_proba基本上一样，只是把结果给做了log()处理。<br>


# RandomForest调参实例

In [6]:

#导入需要的库
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pylab as plt

In [15]:
#导入数据，顺便看看数据的类别分布
train= pd.read_csv('M:/datasets/train_modified.csv',encoding='unicode_escape')
train.head()

,Disbursed,Existing_EMI,ID,Loan_Amount_Applied,Loan_Tenure_Applied,Monthly_Income,Var4,Var5,Age,EMI_Loan_Submitted_Missing,...,Var2_2,Var2_3,Var2_4,Var2_5,Var2_6,Mobile_Verified_0,Mobile_Verified_1,Source_0,Source_1,Source_2
0,0,0.0,ID000002C20,300000,5,20000,1,0,37,1,...,0,0,0,0,1,1,0,1,0,0
1,0,0.0,ID000004E40,200000,2,35000,3,13,30,0,...,0,0,0,0,1,0,1,1,0,0
2,0,0.0,ID000007H20,600000,4,22500,1,0,34,1,...,0,0,0,0,0,0,1,0,0,1
3,0,0.0,ID000008I30,1000000,5,35000,3,10,28,1,...,0,0,0,0,0,0,1,0,0,1
4,0,25000.0,ID000009J40,500000,2,100000,3,17,31,1,...,0,0,0,0,0,0,1,0,0,1


In [16]:
target='Disbursed' # Disbursed的值就是二元分类的输出
IDcol= 'ID'
train['Disbursed'].value_counts()

0    19680
1      320
Name: Disbursed, dtype: int64

In [17]:
#接着选择好样本特征和类别输出，样本特征为除去ID和输出类别的列
x_columns = [x for x in train.columns if x not in [target,IDcol]]
X = train[x_columns]
y = train['Disbursed']

In [19]:
#不管任何参数，都用默认的，拟合下数据看看
rf0 = RandomForestClassifier(oob_score=True, random_state=10)
rf0.fit(X,y)
print(rf0.oob_score_)
y_predprob = rf0.predict_proba(X)[:,1]
print("AUC Score (Train): %f" % metrics.roc_auc_score(y,y_predprob))
#输出如下：0.98005  AUC Score (Train): 0.999833
#可见袋外分数已经很高（理解为袋外数据作为验证集时的准确率，也就是模型的泛化能力），
#而且AUC分数也很高（AUC是指从一堆样本中随机抽一个，抽到正样本的概率比抽到负样本的概率 大的可能性）。
#相对于GBDT的默认参数输出，RF的默认参数拟合效果对本例要好一些。


d:\python\envs\jupyterpy36\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.98005
AUC Score (Train): 0.999833


d:\python\envs\jupyterpy36\lib\site-packages\sklearn\ensemble\forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
d:\python\envs\jupyterpy36\lib\site-packages\sklearn\ensemble\forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


##### best_score_ ：最佳模型下的分数
##### best_params_ ：最佳模型参数
~grid_scores_ ：模型不同参数下交叉验证的平均分~
##### cv_results_ ： 具体用法模型不同参数下交叉验证的结果
##### best_estimator_ : 最佳分类器之所以出现以上问题

In [22]:
#首先对n_estimators进行网格搜索
param_test1= {'n_estimators':range(10,71,10)}

gsearch1= GridSearchCV(
    estimator = RandomForestClassifier(
        min_samples_split=100,
        min_samples_leaf=20,
        max_depth=8,
        max_features='sqrt' ,
        random_state=10
    ),
    param_grid =param_test1, scoring='roc_auc',cv=5
                      )
gsearch1.fit(X,y)

# gsearch1.cv_results_,
# gsearch1.best_params_, 
# gsearch1.best_score_
# grid_scores_在sklearn0.20版本中已被删除，取而代之的是cv_results_。



GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=8, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=20, min_samples_split=100,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=10, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': range(10, 71, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)